In [1]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk

import zipfile
from read_roi import read_roi_zip 

from  scipy import ndimage

from skimage.io import imread, imshow
from skimage import *
import skimage

import pandas as pd
from PIL import *

###########################
def win_scale(data, wl, ww, dtype, out_range):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    
    data_new = np.empty(data.shape, dtype=np.double)
    data_new.fill(out_range[1]-1)
    
    data_new[data <= (wl-ww/2.0)] = out_range[0]
    
    data_new[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))] = ((data[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))]-(wl-0.5))/(ww-1.0)+0.5)*(out_range[1]-out_range[0])+out_range[0]
    
    data_new[data > (wl+ww/2.0)] = out_range[1]-1
    
    return data_new.astype(dtype)

In [2]:
pwd

'/home/ubuntu/gcubme4/Workspace/JW_Seo/DVT_detection/blood_vessel/code'

In [4]:
xlsx = pd.read_excel('../../local/data/dvt_list.xlsx')

In [6]:
p_numpath= sorted(glob.glob('../../local/data/dvt/*'))
print(p_numpath[0])

../../local/data/dvt/01013843


In [7]:
p_numStr=[]
p_numInt=[]
for pn in range(len(p_numpath)):
    p_numStr.append(p_numpath[pn][p_numpath[pn].rindex('/')+1:])
    p_numInt.append(int(p_numpath[pn][p_numpath[pn].rindex('/')+1:]))
print(p_numStr[0])
print(p_numInt[0])
print(len(p_numpath),len(p_numStr),len(p_numInt))

01013843
1013843
200 200 200


In [8]:
print(xlsx['ID'][0])
print(xlsx)

01013843
      No          Study Date        ID Name DVT  \
0     71 2019-11-03 14:34:20  01013843  김옥희   O   
1    104 2019-11-21 15:39:27  01203803  윤용길   O   
2     17 2019-09-07 01:43:30  01537731  김판금   O   
3      1 2019-08-22 00:37:34  01645613  이춘자   O   
4     82 2019-11-09 19:29:09  01727253  정순덕   O   
..   ...                 ...       ...  ...  ..   
207  203 2020-02-19 23:51:49  41011830  이우성   O   
208  214 2020-03-03 00:24:51  41016835  남선우   O   
209  221 2020-03-07 18:49:11  41022878  유재경   O   
210  215 2020-03-03 23:26:25  41023718  안정흠   O   
211  220 2020-03-05 15:36:07  41030879  구운서   O   

                                                    위치   WW   WL  \
0                       Lt CFV, Lt SFV, Lt PV, Lt calf  380   80   
1                                              Rt calf  330  100   
2                                              Rt calf  450  100   
3                                              Rt calf  450   45   
4                                     

In [9]:
p_ids = []
ww_list = []
wl_list = []
for ids in range(len(xlsx)):
    p_ids.append(str(xlsx['ID'][ids]))
    ww_list.append(xlsx['WW'][ids])
    wl_list.append(xlsx['WL'][ids])
print(len(p_ids), len(ww_list), len(wl_list))    

212 212 212


In [15]:
print(type(p_ids[30]))
print(p_ids)

<class 'str'>
['01013843', '01203803', '01537731', '01645613', '01727253', '01827453', '02602503', '03867833', '04425393', '04544953', '04566843', '04765293', '05113703', '05287153', '06362903', '06610173', '06917653', '08187183', '08222013', '08587193', '08637023', '08671413', '09033493', '09154913', '09245063', '09409533', '09902573', '10471963', '10738863', '10863183', '11490283', '11667283', '11918903', '12019383', '12457443', '12628693', '13505103', '13719443', '13720323_3', '13720323_2', '13720323', '13744443', '13986173', '13997853', '14249913', '14462033', '14597783', '16278373', '16335563', '16410373', '16672183', '16811823', '17126793', '17551543', '18145093', '18354683', '18493913', '18523913', '18539623', '18791103', '18890993', '18890993_2', '19152283', '19418913', '19445983', '19447203', '19495593', '19584543_2', '19584543', '19938653', '20275533', '20853143', '20876923', '20954723', '21538113', '21562533', '21792963', '21792963_2', '22069143', '22073603', '22636343', '22

In [10]:
print(p_ids.index(p_numStr[45]))
print(p_ids[p_ids.index(p_numStr[45])], p_numStr[45])
ww=[]
wl=[]
for tt in range(len(p_numStr)):
    if p_numStr[tt] not in p_ids:
        print(p_numStr[tt], 'files not in list')
    else:
        print('number:', tt, 'match:', p_ids[p_ids.index(p_numStr[tt])], p_numStr[tt])
        if p_ids[p_ids.index(p_numStr[tt])] != p_numStr[tt]:
            print('No matching')
        else:
            ww.append(ww_list[p_ids.index(p_numStr[tt])])
            wl.append(wl_list[p_ids.index(p_numStr[tt])])

49
16410373 16410373
number: 0 match: 01013843 01013843
number: 1 match: 01203803 01203803
number: 2 match: 01537731 01537731
number: 3 match: 01645613 01645613
number: 4 match: 01727253 01727253
number: 5 match: 02602503 02602503
number: 6 match: 03867833 03867833
number: 7 match: 04544953 04544953
number: 8 match: 04566843 04566843
number: 9 match: 04765293 04765293
number: 10 match: 05113703 05113703
number: 11 match: 05287153 05287153
number: 12 match: 06362903 06362903
number: 13 match: 06610173 06610173
number: 14 match: 06917653 06917653
number: 15 match: 08187183 08187183
number: 16 match: 08222013 08222013
number: 17 match: 08587193 08587193
number: 18 match: 08637023 08637023
number: 19 match: 08671413 08671413
number: 20 match: 09154913 09154913
number: 21 match: 09245063 09245063
number: 22 match: 09409533 09409533
number: 23 match: 09902573 09902573
number: 24 match: 10738863 10738863
number: 25 match: 10863183 10863183
number: 26 match: 11490283 11490283
number: 27 match:

In [17]:
print(len(p_numpath), len(ww), len(wl))
dcmfiles= glob.glob(p_numpath[0]+'/*.dcm')
print(dcmfiles[0], dcmfiles[0][dcmfiles[0].rindex('/') + 1 :dcmfiles[0].rindex('.dcm')])

200 200 200
../../local/data/dvt/01013843/01013843_0000.dcm 01013843_0000


In [19]:
imgpath='../../defaultImg'

In [22]:
for df in range(len(p_numpath)):
    os.mkdir(imgpath+ '/' + p_numStr[df])
    png_path= imgpath+ '/' + p_numStr[df]
    dcmfiles= glob.glob(p_numpath[df]+'/*.dcm')
    # print(dcmfiles)
    for dwh in range(len(dcmfiles)):

        dcmname= dcmfiles[dwh][dcmfiles[0].rindex('/') + 1 :dcmfiles[0].rindex('.dcm')]

        dcm_dcm = sitk.ReadImage(dcmfiles[dwh])
        dcm_img = sitk.GetArrayFromImage(dcm_dcm)[0]

        dcm_scaled = win_scale(dcm_img, wl[df], ww[df], np.uint8, (0, 255))

#         plt.figure(figsize=(10,10))
#         plt.imshow(dcm_scaled, cmap='gray')
        
        io.imsave(png_path+'/'+ dcmname+'.png', dcm_scaled)


/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0257.png is a low contrast image
/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0258.png is a low contrast image
/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0259.png is a low contrast image
/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0260.png is a low contrast image
/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0261.png is a low contrast image
/home/ubuntu/anaconda3/envs/zzaem1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: ../../defaultImg/01203803/01203803_0262.png is a